In [84]:
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import cv2

final_img=[]

#pseudokod za: pl.wikipedia.org

# x1 , y1 - współrzędne początku odcinka
# x2 , y2 - współrzędne końca odcinka
def BresenhamLine(x1,y1,x2,y2):
    #zmienne pomocnicze
    d=-1
    dx=-1
    dy=-1
    ai=-1
    bi=-1
    xi=-1
    yi=-1
    x=x1
    y=y1
    
    vertexes_x=[]
    vertexes_y=[]
    
    # ustalenie kierunku rysowania
    if(x1 < x2):
        xi = 1
        dx = x2 - x1
    else:
        xi = -1
        dx = x1 - x2
     
    # ustalenie kierunku rysowania
    if (y1 < y2):
        yi = 1
        dy = y2 - y1
    else:
        yi = -1
        dy = y1 - y2

     # pierwszy piksel
    vertexes_x.append(x)
    vertexes_y.append(y)
    
     # oś wiodąca OX
    if (dx > dy):
        ai = (dy - dx) * 2
        bi = dy * 2
        d = bi - dx
        #pętla po kolejnych x
        while (x != x2):
            #test współczynnika
            if (d >= 0):
                x += xi
                y += yi
                d += ai
            else:
                d += bi
                x += xi
                
            vertexes_x.append(x)
            vertexes_y.append(y)
     #oś wiodąca OY
    else:
        ai = ( dx - dy ) * 2
        bi = dx * 2
        d = bi - dy
        #pętla po kolejnych y
        while (y != y2):
             #test współczynnika
            if (d >= 0):
                x += xi
                y += yi
                d += ai
            else:
                d += bi
                y += yi
             
            vertexes_x.append(x)
            vertexes_y.append(y)
    return vertexes_x,vertexes_y
            

def radon_transform(filename, num_of_detectors,alpha_delta,offset,enable_filter):
    #load image
    img=io.imread(filename,as_gray=True)
    #cv2.normalize(img, img, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

    #get width and radius
    width,height=img.shape[0],img.shape[1]
    
    radius=min(width,height)/2
    img_centre_x,img_centre_y=width/2,height/2
    
    
    print("width=",width,"height=",height,"radius=",radius)

    #calculate list of angles based on alpha_delta
    angles=np.arange(0,360,alpha_delta)

    #calculate offsets
    offsets=np.linspace(-offset/2,offset/2,num_of_detectors)

    partial_output=[]
    sinogram=[]
    
    kernel_size=21
    kernel=np.zeros(kernel_size)
    center = kernel_size // 2
    for k in range(kernel_size):
        kernel[k] = ( 0 if (k-center)%2==0 else ((-4/np.pi**2)/((k-center)**2)))
    kernel[center] = 1

    for angle in angles:
        #calculate start end points of rays
        start_x_coords,start_y_coords,end_x_coords,end_y_coords=[],[],[],[]
        for k in range(num_of_detectors):
            start_x_coord=radius*np.cos(np.deg2rad(angle))+img_centre_x
            start_y_coord=radius*np.sin(np.deg2rad(angle))+img_centre_y

            end_x_coords.append(radius*np.cos(np.deg2rad(angle+offsets[k]+180))+img_centre_x)
            end_y_coords.append(radius*np.sin(np.deg2rad(angle+offsets[k]+180))+img_centre_y)

        lines_coords=[]

        #apply Bresenham algorithm
        for line in range(num_of_detectors):
            lines_coords.append(BresenhamLine(int(start_x_coord),int(start_y_coord),
                                              int(end_x_coords[line]),int(end_y_coords[line])))


        #calculate brightness of each ray
        lines_brightnesses=[]

        for line in lines_coords:
            res=0
            for i in range(len(line[0])):
                if(line[0][i]<width and line[1][i]<height): 
                    res+=img[line[0][i],line[1][i]]
            lines_brightnesses.append(res/len(line[0])) #normalization
    
        #filter
        if(enable_filter): lines_brightnesses=np.convolve(lines_brightnesses, kernel, mode='same')
            
        sinogram.append(lines_brightnesses)
        
        

        #x -> lines_coords[i][0]
        #y -> lines_coords[i][1]
        #brigtness -> lines_brightnesses[i]
        output=np.zeros((width, height))



        for i in range(len(lines_coords)):
            for j in range(len(lines_coords[i][0])):
                x=lines_coords[i][0][j]
                y=lines_coords[i][1][j]

                if(x<width and y<height): 
                    output[x][y]+=lines_brightnesses[i]
        
        #normalization
        output/=len(lines_coords)
    
        #result of this iteration
        partial_output.append(output)
        
        print(".",end="")       


    print("\nstep 1/2 DONE")
    
    #calculate result of all iterations
    partial_avg_output=[np.zeros((width, height)) for i in range(len(angles))]
    
    for i in range(width):
        for j in range(height):
            suma=0
            for iteration in range(0,len(angles)):
                suma+=partial_output[iteration][i][j]
                partial_avg_output[iteration][i][j]=suma/(iteration+1)
        
        if(i%10==0):print(".",end="")
    
    #normalize final image and result of each iteration
    for i in range(len(partial_avg_output)):
        cv2.normalize(partial_avg_output[i], partial_avg_output[i], alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)   
    
        
    print("\nstep 2/2 DONE")
    
    
    return img,sinogram,partial_avg_output

def show_result(img,sinogram,partial_avg_output,x):
    
    print('Current iteration='+str(x)+" of "+str(len(partial_avg_output)))
    
    #take original image, normalized final output and calc RMSE
    print('RMSE=',np.sqrt(((img-partial_avg_output[x-1])**2).mean()))
    
    plt.figure(figsize=(30,60))
    
    plt.subplot(1,3,1)
    plt.title('Input image')
    plt.imshow(img,'gray')

    plt.subplot(1,3,2)
    plt.title('Sinogram')
    plt.imshow(sinogram[:x],'gray')

    plt.subplot(1,3,3)
    plt.title('Output image')
    plt.imshow(partial_avg_output[x-1],'gray')
    
    plt.show()

    
import pydicom
from pydicom.dataset import Dataset, FileDataset, FileMetaDataset
import datetime

def write_as_dicom(pixel_array,name,patient_id,out_filename,comment):
    meta = Dataset()
    meta.MediaStorageSOPClassUID ='1.2.840.10008.5.1.4.1.1.2'
    meta.MediaStorageSOPInstanceUID = pydicom.uid.generate_uid()
    meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian
    ds = FileDataset(None, {}, preamble=b"\0" * 128)
    ds.file_meta = meta
        
    ds.is_little_endian = True
    ds.is_implicit_VR = False

    ds.SOPInstanceUID = meta.MediaStorageSOPInstanceUID

    ds.PatientName = name
    ds.PatientID = patient_id
    ds.ImageComments = comment


    ds.Modality = "CT"
    ds.SeriesInstanceUID = pydicom.uid.generate_uid()
    ds.StudyInstanceUID = pydicom.uid.generate_uid()
    ds.FrameOfReferenceUID = pydicom.uid.generate_uid()

    ds.BitsStored =16
    ds.BitsAllocated = 16
    ds.SamplesPerPixel = 1
    ds.HighBit = 15

    ds.ImagesInAcquisition = 1
    ds.InstanceNumber = 1

    ds.Rows= pixel_array.shape[0]
    ds.Columns = pixel_array.shape[1]

    ds.ImageType = r"ORIGINAL\PRIMARY\AXIAL"

    ds.PhotometricInterpretation = "MONOCHROME2"
    ds.PixelRepresentation = 0
    
    dt = datetime.datetime.now()
    ds.ContentDate = dt.strftime('%Y%m%d')
    timeStr = dt.strftime('%H%M%S.%f')
    ds.ContentTime = timeStr

    pydicom.dataset.validate_file_meta(ds.file_meta, enforce_standard=True)

    cv2.normalize(pixel_array, pixel_array, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)   
    pixel_array = pixel_array.astype(np.int16)

    ds.PixelData = pixel_array.tobytes()

    ds.save_as(out_filename, write_like_original=False)
    
    print("\033[1m saved! \033[1m")
    

In [85]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual,FloatSlider,IntSlider



def user_IO(filename,detectors,angle_delta,offset,enable_filter):
    img,sinogram,partial_avg_output=radon_transform(filename,detectors,angle_delta,offset,enable_filter)
    
    
    print("\033[1m_________________________________\033[1m")
    print("\033[1m Select desired iteration: \033[1m")
    
    interact(show_result,
             img=fixed(img),
             sinogram=fixed(sinogram),
             partial_avg_output=fixed(partial_avg_output),
             x=widgets.IntSlider(min=1, max=len(partial_avg_output), step=1, value=len(partial_avg_output)))
    
    print('\033[1m Here you can export result to DICOM file: \033[0m')
    
    interact_manual(write_as_dicom,
                pixel_array=fixed(partial_avg_output[-1]),
                name="Pawel Nowak",
                patient_id="12345",
                comment="add comment",
                out_filename="output.dcm"
               )

print("\033[1m CT simulator - type required data below \033[1m")
interact_manual(user_IO,
                enable_filter=False,
                filename="tomograf-zdjecia/Kropka1.jpg",
                detectors=IntSlider(min=1,max=800,value=180),
                angle_delta=FloatSlider(min=0.001,max=50,value=2),
                offset=IntSlider(min=20,max=300,value=180))

 CT simulator - type required data below 


interactive(children=(Text(value='tomograf-zdjecia/Kropka1.jpg', description='filename'), IntSlider(value=180,…

<function __main__.user_IO(filename, detectors, angle_delta, offset, enable_filter)>

# Open DICOM file:

In [86]:
def open_file(filename):
    my_file=pydicom.dcmread(filename)
    print(my_file)
    
    #check bitsstored value
    converted=''
    if(my_file.BitsStored==16): converted=np.frombuffer(my_file.PixelData,dtype=np.uint16)
    elif(my_file.BitsStored==8):converted=np.frombuffer(my_file.PixelData,dtype=np.uint8)
    else: return 
    
    img=np.zeros((int(my_file.Rows),int(my_file.Columns)))
    
    idx=0
    for i in range(int(my_file.Rows)):
        for j in range(int(my_file.Columns)):
            img[i][j]=converted[idx]
            idx+=1
   
    plt.imshow(img,'gray')
    

print("\033[1m Choose DICOM file to open: \033[1m")
interact_manual(open_file,
                filename="output.dcm")

 Choose DICOM file to open: 


interactive(children=(Text(value='output.dcm', description='filename'), Button(description='Run Interact', sty…

<function __main__.open_file(filename)>

# Modify existing DICOM file:

In [87]:
def modify_dicom(filename,name,patient_id,comment):
    my_file=pydicom.dcmread(filename)
    if(name!=''):my_file.PatientName=name
    if(patient_id!=''):my_file.PatientID=patient_id
    if(comment!=''):my_file.ImageComments=comment
    my_file.save_as(filename, write_like_original=False)
    print("file changed!")
    

interact_manual(modify_dicom,
                filename='output.dcm',
                name="",
                patient_id="",
                comment="",
               )

interactive(children=(Text(value='output.dcm', description='filename'), Text(value='', description='name'), Te…

<function __main__.modify_dicom(filename, name, patient_id, comment)>